In [ ]:
import torch
import torch.nn as nn
import torchvision as tv
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchsummary import summary
import pandas as pd
import gc
from torchvision import transforms, datasets

In [ ]:
transform = tv.transforms.Compose([

                                        tv.transforms.Resize((224, 224)),
                                        tv.transforms.ToTensor()])

In [ ]:
train_dataset = datasets.ImageFolder('hymenoptera_data/train', transform)
test_dataset = datasets.ImageFolder('hymenoptera_data/val', transform)

In [ ]:

def show_images(images):
    plt.figure(figsize=(10, 10))
    for i, image in enumerate(images):
      numpy_image = image.numpy()
      plt.subplot(5, 5, i+1)
      # Transpose the numpy array to match the Matplotlib format
      matplotlib_image = numpy_image.transpose(1,2,0)

      # Display the image using Matplotlib
      plt.imshow(matplotlib_image)

    plt.show()

In [ ]:
sample_images = [train_dataset[i][0] for i in range(5)]
show_images(sample_images)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
def train(model, train_iter, test_iter,trainer,epochs=10):

    l_arr = []
    model.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    model.train()

    for epoch in range(epochs):
      train_l_sum,n,train_acc_sum = 0.0, 0,0.0
      for i,(X,y) in enumerate(train_iter):

        X,y = X.to(device),y.to(device)
        trainer.zero_grad()
        y_pred = model(X)
        l = loss(y_pred,y)
        l.backward()
        trainer.step()
        train_l_sum  += l.item()
        train_acc_sum += (y_pred.argmax(axis=1)==y).sum().item()
        n += y.shape[0]
        torch.cuda.empty_cache()
        #if epoch % 2:
         # print(f"""Batch {i} LOSS =  {train_l_sum/n} AUC = {train_acc_sum/n} """)
     # if epoch % 2 :
     #   test_acc = AUC(model,test_iter)
     #   print(f"""----------------AUC test = {test_acc}------------------------ """)
        del X
        del y
      l_arr.append(train_l_sum/n)
      gc.collect()
    return pd.DataFrame({'epoch':list(range(epochs)),'Loss':l_arr})

In [ ]:
len(train_dataset)

In [ ]:
# Resnet

In [ ]:
model = tv.models.resnet18(pretrained=False)

In [ ]:
targets = [train_dataset[i][1] for i in range(len(train_dataset))]

In [ ]:
model.classifier = nn.Linear(in_features=2208, out_features=len(targets))

In [ ]:
df = pd.DataFrame({'target':targets})


In [ ]:
df['count'] = 1
df.groupby('target').count()

In [ ]:
BATCH_SIZE = 10
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model.classifier = nn.Linear(in_features=2208, out_features=len(targets))

In [ ]:

params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
res_results = train(model, train_iter, test_iter, trainer, 10)

In [ ]:
#vgg

In [ ]:
transform = tv.transforms.Compose([
      tv.transforms.Grayscale(3),
    tv.transforms.Resize((32,32)),
    tv.transforms.ToTensor()
])

In [ ]:
train_dataset = datasets.ImageFolder('hymenoptera_data/train', transform)
test_dataset = datasets.ImageFolder('hymenoptera_data/val', transform)

In [ ]:
model = tv.models.vgg16(pretrained=False)

In [ ]:
model.classifier = nn.Linear(in_features=25088, out_features=len(targets))

In [ ]:

params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)


In [ ]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
vgg_results = train(model, train_iter, test_iter, trainer, 10)

In [ ]:
# Inception V3

In [ ]:
transform = tv.transforms.Compose([
    tv.transforms.Resize((299, 299)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
])


train_dataset = datasets.ImageFolder('hymenoptera_data/train', transform)
test_dataset = datasets.ImageFolder('hymenoptera_data/val', transform)

In [ ]:
BATCH_SIZE = 10
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = tv.models.inception_v3(pretrained=False)

In [ ]:
model.classifier = nn.Linear(model.fc.in_features, len(targets))


In [ ]:

params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [ ]:
def train_1(model, train_iter, test_iter,trainer,epochs=10):
  l_arr = []
  model.to(device)
  loss = nn.CrossEntropyLoss(reduction='sum')
  model.train()

  for epoch in range(epochs):
    train_l_sum,n,train_acc_sum = 0.0, 0,0.0
    for i,(X,y) in enumerate(train_iter):


      X,y = X.to(device),y.to(device)
      trainer.zero_grad()
      _,y_pred = model(X)
      l = loss(y_pred,y)
      l.backward()
      trainer.step()
      train_l_sum  += l.item()
      train_acc_sum += (y_pred.argmax(axis=1)==y).sum().item()
      n += y.shape[0]
      torch.cuda.empty_cache()
     # if i % 2:
     #   print(f"""Batch {i} LOSS =  {train_l_sum/n} AUC = {train_acc_sum/n} """)
    #if epoch % 2 :
    #  test_acc = AUC(model,test_iter)
    #  print(f"""----------------AUC test = {test_acc}------------------------ """)

    l_arr.append(train_l_sum/n)
  return pd.DataFrame({'epoch':list(range(epochs)),'Loss':l_arr})
BATCH_SIZE = 10
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)
inception_results = train_1(model, train_iter, test_iter, trainer, 10)

In [ ]:
#DENSENET

In [ ]:
model = tv.models.densenet161(pretrained=False)

In [ ]:
transform_ = tv.transforms.Compose([
    tv.transforms.Resize((32, 32)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
])

In [ ]:
train_dataset = datasets.ImageFolder('hymenoptera_data/train', transform)
test_dataset = datasets.ImageFolder('hymenoptera_data/val', transform)

In [ ]:
BATCH_SIZE = 10
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model.classifier = nn.Linear(in_features=2208, out_features=len(targets))

In [ ]:

params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)


In [ ]:
BATCH_SIZE = 10
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)
dense_results = train(model, train_iter, test_iter, trainer, 10)

In [ ]:
# Results

In [ ]:
import matplotlib.pyplot as plt

# Plot
plt.plot(res_results['epoch'], res_results['Loss'],label='Resnet')
#plt.plot(vgg_results['epoch'], vgg_results['Loss'],label='VGG')
plt.plot(inception_results ['epoch'], inception_results ['Loss'],label='Inception')
plt.plot(dense_results ['epoch'],dense_results  ['Loss'],label='DenseNet')
plt.legend()
plt.show()